In [4]:
from centerline import CenterLine, smooth_mask, iou
from scipy import io as sio
from glob import glob
from tqdm.notebook import tqdm
from skimage import io, morphology, img_as_float, filters, exposure
import numpy as np
import math
import pickle

### Testing set 0

In [5]:
### Process a batch of network output
# raw_input_fnames = glob('evaluations/pilot/aug_output/resTe/*_I.png') # raw input
nw_fnames = glob('evaluations/pilot/41nw_output/processed/skeletonize/0/*.tif') # nw masks
aug_fnames = glob('evaluations/pilot/50_output/processed/skeletonize/0/*.tif') # aug masks
target_fnames = glob('evaluations/pilot/target/0/*_Ltrue.png') # truth masks
mat_fnames = glob('evaluations/pilot/ctFIRE_out/0/*.tif') # ctFIRE masks
ridge_fnames = glob('evaluations/pilot/ridge_detector/0/*.tif') # ctFIRE masks

ridge_feats = []
mat_feats = []
nw_feats = []
aug_feats = []
truth_feats = []

### generate from output masks
for i in tqdm(range(len(nw_fnames))):
    nw_fname = nw_fnames[i]
    aug_fname = aug_fnames[i]
    target_fname = target_fnames[i]
    mat_fname = mat_fnames[i]
    ridge_fname = ridge_fnames[i]

    ### ground truth
    centerline = CenterLine(centerline_image=io.imread(target_fname))
    centerline.compute_fiber_feats() 
    truth_feats.append(centerline.feats)

    ## nw
    centerline_nw = CenterLine(centerline_image=io.imread(nw_fname))
    centerline_nw.compute_fiber_feats() 
    nw_feats.append(centerline_nw.feats)

    ### aug
    centerline_aug = CenterLine(centerline_image=io.imread(aug_fname))
    centerline_aug.compute_fiber_feats() 
    aug_feats.append(centerline_aug.feats)

    ### ridge detector
    centerline_ridge = CenterLine(centerline_image=io.imread(ridge_fname))
    centerline_ridge.compute_fiber_feats() 
    ridge_feats.append(centerline_ridge.feats)

    ### ctFIRE
    centerline_mat = CenterLine(centerline_image=io.imread(mat_fname))
    centerline_mat.compute_fiber_feats() 
    mat_feats.append(centerline_mat.feats)

0it [00:00, ?it/s]

In [6]:
### read saved results
with open('evaluations/ridge_feats_0.pkl', 'rb') as f:
    ridge_feats = pickle.load(f)
    
with open('evaluations/mat_feats_0.pkl', 'rb') as f:
    mat_feats = pickle.load(f)

with open('evaluations/nw_feats_0.pkl', 'rb') as f:
    nw_feats = pickle.load(f)

with open('evaluations/truth_feats_0.pkl', 'rb') as f:
    truth_feats = pickle.load(f)

with open('evaluations/aug_feats_0.pkl', 'rb') as f:
    aug_feats = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'evaluations/ridge_feats_0.pkl'

In [ ]:
# ### Process one kind of output
# raw_input_fnames = glob('evaluations/pilot/aug_output/resTe/*_I.png') # raw input
# sk_fnames = glob('evaluations/pilot/network_output/processed/skeletonize/*.tif') # nw masks

# sk_feats = []

# ### generate from output masks
# for i in tqdm(range(len(raw_input_fnames))):
#     sk_fname = sk_fnames[i]

#     ### ground truth
#     centerline = CenterLine(centerline_image=io.imread(sk_fname))
#     centerline.compute_fiber_feats() 
#     sk_feats.append(centerline.feats)

In [ ]:
feat_type = {}
for feat in truth_feats:
    for k, v in feat.items():
        try:
            feat_type[k].append(v)
        except:
            feat_type[k] = [v]

feat_bound = {}
for k, v in feat_type.items():
    feat_bound[k] = (np.percentile(v, 0), np.percentile(v, 100))

In [ ]:
def feats_diff(feats_in, feats_ref, bound=None, abs_val=True):
    diff = []
    cir_dist = lambda x, y, r=math.pi: min(r - abs(x - y), abs(x - y))
    normalize = lambda x, k, bound : exposure.rescale_intensity(np.array([x]), in_range=(bound[k][0], bound[k][1]), out_range=(0, 1))[0]
    for k, v in feats_in.items():
        if bound:
            feat_in = normalize(feats_in[k], k, bound) 
            feat_ref = normalize(feats_ref[k], k, bound)
        else:
            feat_in = feats_in[k]
            feat_ref = feats_ref[k]
        if k == 'cir_mean':
            diff.append(cir_dist(feats_in[k], feats_ref[k]))
        elif k == 'density':
            ratio, U, I = iou(feat_in, feat_ref)
            diff.append(ratio)
        else:
            if abs_val: 
                diff.append(abs(feat_in - feat_ref))
            else: 
                diff.append(feat_in - feat_ref)
    return diff

In [ ]:
ridge_diffs = []
mat_diffs = []
nw_diffs = []
aug_diffs = []
for i in range(len(ridge_feats)):
    ridge_diffs.append(feats_diff(ridge_feats[i], truth_feats[i]))
    mat_diffs.append(feats_diff(mat_feats[i], truth_feats[i]))
    nw_diffs.append(feats_diff(nw_feats[i], truth_feats[i]))
    aug_diffs.append(feats_diff(aug_feats[i], truth_feats[i]))
ridge_diffs = np.stack(ridge_diffs)
mat_diffs = np.stack(mat_diffs)
nw_diffs = np.stack(nw_diffs)
aug_diffs = np.stack(aug_diffs)

max_diff = (math.pi, 
feat_bound['cir_var'][1]-feat_bound['cir_var'][0], 
feat_bound['len_mean'][1]-feat_bound['len_mean'][0], 
feat_bound['len_var'][1]-feat_bound['len_var'][0], 
feat_bound['waviness'][1]-feat_bound['waviness'][0], 
feat_bound['intensity'][1]-feat_bound['intensity'][0])

normalize_error = lambda diff, max_diff: [diff[i]/max_diff[i] if i<6 else diff[i] for i in range(7)]
norm_ridge_diffs = np.array([normalize_error(i, max_diff) for i in ridge_diffs])
norm_mat_diffs = np.array([normalize_error(i, max_diff) for i in mat_diffs])
norm_nw_diffs = np.array([normalize_error(i, max_diff) for i in nw_diffs])
norm_aug_diffs = np.array([normalize_error(i, max_diff) for i in aug_diffs])

summary = lambda name, diffs: print(f'{name} normalized error : cir_mean {diffs[0]:.4f} cir_var {diffs[1]:.4f} len_mean {diffs[2]:.4f} len_var {diffs[3]:.4f} waviness {diffs[4]:.4f} intensity {diffs[5]:.4f} || IoU: {diffs[6]:.4f}')
summary('Ridge', np.mean(norm_ridge_diffs, 0))
summary('ctFIRE',np.mean(norm_mat_diffs, 0))
summary('Network',np.mean(norm_nw_diffs, 0))
summary('Aug',np.mean(norm_aug_diffs, 0))

Ridge normalized error : cir_mean 0.0584 cir_var 0.1649 len_mean 0.0955 len_var 0.0877 waviness 0.2997 intensity 0.1604 || IoU: 0.6704
ctFIRE normalized error : cir_mean 0.0597 cir_var 0.1172 len_mean 0.0718 len_var 0.1033 waviness 0.2278 intensity 0.0945 || IoU: 0.6778
Network normalized error : cir_mean 0.0567 cir_var 0.0728 len_mean 0.0555 len_var 0.0712 waviness 0.1288 intensity 0.0628 || IoU: 0.7414
Aug normalized error : cir_mean 0.0527 cir_var 0.0781 len_mean 0.0541 len_var 0.0617 waviness 0.1412 intensity 0.0588 || IoU: 0.7437


Ridge normalized error : cir_mean 0.0584 cir_var 0.1649 len_mean 0.0955 len_var 0.0877 waviness 0.2997 intensity 0.1604 || IoU: 0.6704  
ctFIRE normalized error : cir_mean 0.0597 cir_var 0.1172 len_mean 0.0718 len_var 0.1033 waviness 0.2278 intensity 0.0945 || IoU: 0.6778  
Network normalized error : cir_mean 0.0505 cir_var 0.0698 len_mean 0.0487 len_var 0.0694 waviness 0.1283 intensity 0.0647 || IoU: 0.7352  
Aug normalized error : cir_mean 0.0481 cir_var 0.0737 len_mean 0.0510 len_var 0.0665 waviness 0.1281 intensity 0.0699 || IoU: 0.7391  

Ridge normalized error : cir_mean 0.0579 cir_var 0.1652 len_mean 0.0750 len_var 0.0720 waviness 0.3269 intensity 0.1839 || IoU: 0.6585  
ctFIRE normalized error : cir_mean 0.0624 cir_var 0.1274 len_mean 0.0680 len_var 0.1006 waviness 0.2498 intensity 0.1233 || IoU: 0.6561  
Network normalized error : cir_mean 0.0508 cir_var 0.0871 len_mean 0.0391 len_var 0.0577 waviness 0.1540 intensity 0.0966 || IoU: 0.7086  

Aug-v1 normalized error : cir_mean 0.0514 cir_var 0.0796 len_mean 0.0462 len_var 0.0680 waviness 0.1520 intensity 0.1163 || IoU: 0.7023 

Aug-v2 normalized error : cir_mean 0.0503 cir_var 0.0905 len_mean 0.0450 len_var 0.0697 waviness 0.1573 intensity 0.1299 || IoU: 0.7026

Aug-v3 normalized error : cir_mean 0.0515 cir_var 0.0845 len_mean 0.0517 len_var 0.0646 waviness 0.1442 intensity 0.1110 || IoU: 0.7000

Aug-v4 normalized error : cir_mean 0.0511 cir_var 0.0814 len_mean 0.0461 len_var 0.0687 waviness 0.1476 intensity 0.1182 || IoU: 0.7041

Aug-v4: normalized error : cir_mean 0.0457 cir_var 0.0730 len_mean 0.0461 len_var 0.0646 waviness 0.1437 intensity 0.1031 || IoU: 0.7042

Aug-joon normalized error : cir_mean 0.0532 cir_var 0.0828 len_mean 0.0446 len_var 0.0693 waviness 0.1447 intensity 0.1520 || IoU: 0.6804


In [ ]:
with open('evaluations/ridge_feats_0.pkl', 'wb') as f:
    pickle.dump(ridge_feats, f)
        
with open('evaluations/mat_feats_0.pkl', 'wb') as f:
    pickle.dump(mat_feats, f)

with open('evaluations/nw_feats_0.pkl', 'wb') as f:
    pickle.dump(nw_feats, f)

with open('evaluations/aug_feats_0.pkl', 'wb') as f:
    pickle.dump(aug_feats, f)

with open('evaluations/truth_feats_0.pkl', 'wb') as f:
    pickle.dump(truth_feats, f)

In [ ]:
diffs_pack = np.dstack([norm_ridge_diffs, norm_mat_diffs, norm_nw_diffs, norm_aug_diffs])
pack = np.concatenate((diffs_pack[:, :6, :], (1-diffs_pack[:, -1, :])[:, None, :]), 1)
sample_scores = []
for i in range(pack.shape[0]):
    stack = pack[i, :, :]
    ranks =  np.argsort(stack, 1).argsort()
    scores = np.mean(ranks, 0)
    sample_scores.append(scores)
sample_scores = np.vstack(sample_scores)
rank_score = np.mean(sample_scores, 0)
print(f'Average rank score: Ridge {rank_score[0]:.4f} ctFIRE {rank_score[1]:.4f} Network {rank_score[2]:.4f} Aug {rank_score[3]:.4f}')

Average rank score: Ridge 2.0195 ctFIRE 1.7862 Network 1.1213 Aug 1.0730


## Phase 2

In [ ]:
### Process a batch of network output
# raw_input_fnames = glob('evaluations/pilot/aug_output/resTe/*_I.png') # raw input
nw_fnames = glob('evaluations/pilot/41nw_output/processed/skeletonize/1/*.tif') # nw masks
aug_fnames = glob('evaluations/pilot/41aug_output/processed/skeletonize/1/*.tif') # aug masks
target_fnames = glob('evaluations/pilot/target/1/*_Ltrue.png') # truth masks
mat_fnames = glob('evaluations/pilot/ctFIRE_out/1/*.tif') # ctFIRE masks
ridge_fnames = glob('evaluations/pilot/ridge_detector/1/*.tif') # ctFIRE masks

ridge_feats = []
mat_feats = []
nw_feats = []
aug_feats = []
truth_feats = []

### generate from output masks
for i in tqdm(range(len(nw_fnames))):
    nw_fname = nw_fnames[i]
    aug_fname = aug_fnames[i]
    target_fname = target_fnames[i]
    mat_fname = mat_fnames[i]
    ridge_fname = ridge_fnames[i]

    ### ground truth
    centerline = CenterLine(centerline_image=io.imread(target_fname))
    centerline.compute_fiber_feats() 
    truth_feats.append(centerline.feats)

    ### nw
    centerline_nw = CenterLine(centerline_image=io.imread(nw_fname))
    centerline_nw.compute_fiber_feats() 
    nw_feats.append(centerline_nw.feats)

    ### aug
    centerline_aug = CenterLine(centerline_image=io.imread(aug_fname))
    centerline_aug.compute_fiber_feats() 
    aug_feats.append(centerline_aug.feats)

    ### ridge detector
    centerline_ridge = CenterLine(centerline_image=io.imread(ridge_fname))
    centerline_ridge.compute_fiber_feats() 
    ridge_feats.append(centerline_ridge.feats)

    ### ctFIRE
    centerline_mat = CenterLine(centerline_image=io.imread(mat_fname))
    centerline_mat.compute_fiber_feats() 
    mat_feats.append(centerline_mat.feats)

  0%|          | 0/60 [00:00<?, ?it/s]

In [ ]:
ridge_diffs = []
mat_diffs = []
nw_diffs = []
aug_diffs = []
for i in range(len(ridge_feats)):
    ridge_diffs.append(feats_diff(ridge_feats[i], truth_feats[i]))
    mat_diffs.append(feats_diff(mat_feats[i], truth_feats[i]))
    nw_diffs.append(feats_diff(nw_feats[i], truth_feats[i]))
    aug_diffs.append(feats_diff(aug_feats[i], truth_feats[i]))
ridge_diffs = np.stack(ridge_diffs)
mat_diffs = np.stack(mat_diffs)
nw_diffs = np.stack(nw_diffs)
aug_diffs = np.stack(aug_diffs)

feat_type = {}
for feat in truth_feats:
    for k, v in feat.items():
        try:
            feat_type[k].append(v)
        except:
            feat_type[k] = [v]

feat_bound = {}
for k, v in feat_type.items():
    feat_bound[k] = (np.percentile(v, 0), np.percentile(v, 100))

max_diff = (math.pi, 
feat_bound['cir_var'][1]-feat_bound['cir_var'][0], 
feat_bound['len_mean'][1]-feat_bound['len_mean'][0], 
feat_bound['len_var'][1]-feat_bound['len_var'][0], 
feat_bound['waviness'][1]-feat_bound['waviness'][0], 
feat_bound['intensity'][1]-feat_bound['intensity'][0])

normalize_error = lambda diff, max_diff: [diff[i]/max_diff[i] if i<6 else diff[i] for i in range(7)]
norm_ridge_diffs = np.array([normalize_error(i, max_diff) for i in ridge_diffs])
norm_mat_diffs = np.array([normalize_error(i, max_diff) for i in mat_diffs])
norm_nw_diffs = np.array([normalize_error(i, max_diff) for i in nw_diffs])
norm_aug_diffs = np.array([normalize_error(i, max_diff) for i in aug_diffs])

summary = lambda name, diffs: print(f'{name} normalized error : cir_mean {diffs[0]:.4f} cir_var {diffs[1]:.4f} len_mean {diffs[2]:.4f} len_var {diffs[3]:.4f} waviness {diffs[4]:.4f} intensity {diffs[5]:.4f} || IoU: {diffs[6]:.4f}')
summary('Ridge', np.mean(norm_ridge_diffs, 0))
summary('ctFIRE',np.mean(norm_mat_diffs, 0))
summary('Network',np.mean(norm_nw_diffs, 0))
summary('Aug',np.mean(norm_aug_diffs, 0))

Ridge normalized error : cir_mean 0.0579 cir_var 0.1652 len_mean 0.0750 len_var 0.0720 waviness 0.3269 intensity 0.1839 || IoU: 0.6585
ctFIRE normalized error : cir_mean 0.0624 cir_var 0.1274 len_mean 0.0680 len_var 0.1006 waviness 0.2498 intensity 0.1233 || IoU: 0.6561
Network normalized error : cir_mean 0.0518 cir_var 0.0767 len_mean 0.0445 len_var 0.0635 waviness 0.1407 intensity 0.0856 || IoU: 0.7127
Aug normalized error : cir_mean 0.0471 cir_var 0.0835 len_mean 0.0438 len_var 0.0560 waviness 0.1510 intensity 0.0819 || IoU: 0.7151


In [ ]:
diffs_pack = np.dstack([norm_ridge_diffs, norm_mat_diffs, norm_nw_diffs, norm_aug_diffs])
pack = np.concatenate((diffs_pack[:, :6, :], (1-diffs_pack[:, -1, :])[:, None, :]), 1)
sample_scores = []
for i in range(pack.shape[0]):
    stack = pack[i, :, :]
    ranks =  np.argsort(stack, 1).argsort()
    scores = np.mean(ranks, 0)
    sample_scores.append(scores)
sample_scores = np.vstack(sample_scores)
rank_score = np.mean(sample_scores, 0)
print(f'Average rank score: Ridge {rank_score[0]:.4f} ctFIRE {rank_score[1]:.4f} Network {rank_score[2]:.4f} Aug {rank_score[3]:.4f}')

Average rank score: Ridge 1.9663 ctFIRE 1.8600 Network 1.1034 Aug 1.0704


In [ ]:
import pickle
with open('evaluations/ridge_feats_1.pkl', 'wb') as f:
    pickle.dump(ridge_feats, f)
        
with open('evaluations/mat_feats_1.pkl', 'wb') as f:
    pickle.dump(mat_feats, f)

with open('evaluations/nw_feats_1.pkl', 'wb') as f:
    pickle.dump(nw_feats, f)

with open('evaluations/aug_feats_1.pkl', 'wb') as f:
    pickle.dump(aug_feats, f)

with open('evaluations/truth_feats_1.pkl', 'wb') as f:
    pickle.dump(truth_feats, f)

In [ ]:
import pickle
with open('evaluations/ridge_diffs.pkl', 'wb') as f:
    pickle.dump(norm_ridge_diffs, f)
        
with open('evaluations/mat_diffs.pkl', 'wb') as f:
    pickle.dump(norm_mat_diffs, f)

with open('evaluations/nw_diffs.pkl', 'wb') as f:
    pickle.dump(norm_nw_diffs, f)

with open('evaluations/aug_diffs.pkl', 'wb') as f:
    pickle.dump(norm_aug_diffs, f)

# with open('evaluations/truth_diffs.pkl', 'wb') as f:
#     pickle.dump(truth_feats, f)

In [ ]:
## read saved results
with open('evaluations/ridge_feats_0.pkl', 'rb') as f:
    ridge_feats = pickle.load(f)
    
with open('evaluations/mat_feats_0.pkl', 'rb') as f:
    mat_feats = pickle.load(f)

with open('evaluations/nw_feats_0.pkl', 'rb') as f:
    nw_feats = pickle.load(f)

with open('evaluations/truth_feats_0.pkl', 'rb') as f:
    truth_feats = pickle.load(f)

with open('evaluations/aug_feats_0.pkl', 'rb') as f:
    aug_feats = pickle.load(f)

In [ ]:
## read saved results
with open('evaluations/ridge_feats_1.pkl', 'rb') as f:
    ridge_feats += pickle.load(f)
    
with open('evaluations/mat_feats_1.pkl', 'rb') as f:
    mat_feats += pickle.load(f)

with open('evaluations/nw_feats_1.pkl', 'rb') as f:
    nw_feats += pickle.load(f)

with open('evaluations/truth_feats_1.pkl', 'rb') as f:
    truth_feats += pickle.load(f)

with open('evaluations/aug_feats_1.pkl', 'rb') as f:
    aug_feats += pickle.load(f)

NameError: name 'ridge_feats' is not defined

In [ ]:
from scipy import stats

In [ ]:
aug_list = []
for item in aug_feats:
    aug_list.append(list(item.values())[:-1])

nw_list = []
for item in nw_feats:
    nw_list.append(list(item.values())[:-1])

In [ ]:
aug_arr = np.array(aug_list)
nw_arr = np.array(nw_list)

In [ ]:
stats.ttest_rel(aug_arr[:, 5], nw_arr[:, 5])

Ttest_relResult(statistic=-2.193551229837585, pvalue=0.03222021342354551)